<a href="https://colab.research.google.com/github/ahmedshuvo969/quora-insincere-questions-classification/blob/master/cosine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Loading & analysis
import the necessary libraries

In [0]:

import re
import os
import nltk
import string
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Uploading file


In [0]:
train=pd.read_csv('/content/train (1).csv')
test=pd.read_csv('test.csv')
test_pred=pd.read_csv('sample_submission.csv')


The distribution of target variable

In [0]:
sns.countplot(train['target'])

Percentage of sincere and insincere questions

In [0]:
len_data = len(train['target'])
count_sin = 0
for i in range(len_data):
    if train['target'][i] == 1:
        count_sin += 1
count_insin = len_data - count_sin
len_text_data = len(test['question_text'])

print('percent of sincere is : ', (count_sin/len_data)*100)
print('percent of insincere is : ', (count_insin/len_data)*100)

# Pre-processing
Check missing or nan value

In [0]:
train.isnull().sum() #absent of missing value

Removing stopwords, cleaning punctuation/html tag/, stemming

In [0]:
#for train corpus
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer 

corpus = []
for i in range(0, len_data):
    review = re.sub('[^a-zA-Z]', ' ', train['question_text'][i])
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)

y_train = []    
for i in range(0, len_data):
    y_train.append(train['target'][i])

In [0]:
#for testing corpus
test_corpus = []
for i in range(0, len_text_data):
    review = re.sub('[^a-zA-Z]', ' ', test['question_text'][i])
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    test_corpus.append(review)


y_test = []    
for i in range(0, len_text_data):
    y_test.append(test_pred['prediction'][i])

separte sincere and insincere 

In [0]:
PosSet = []
NegSet = []
TestSet = []
k = 0
for i in corpus:
    m = i.split(" ")
    if y_train[k] == 1: 
        for j in m:
            PosSet.append(j)
    else:
        for j in m:
            NegSet.append(j)
    k = k+1


PosSet = ' '.join(PosSet)
NegSet = ' '.join(NegSet)



doc = [PosSet, NegSet]


TF-IDF vectorizer

In [0]:

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
matrix1 = vectorizer.fit_transform(doc)



In [0]:
matrix2 = vectorizer.transform(test_corpus)

# Relationship using Cosine Similarity

In [0]:

from sklearn.metrics.pairwise import cosine_similarity
rel = cosine_similarity(matrix2, matrix1)

In [0]:
y_pred = []

for i in range(0, len_text_data):
 
  if rel[i][0]>rel[i][1]:
      y_pred.append(1)
  else:
      y_pred.append(0)

Confusion Matrix

In [0]:

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

Accuracy, Precision, Recall, F1 Score

In [0]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test, y_pred)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, y_pred)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, y_pred)
print('F1 score: %f' % f1)